In [38]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns 
import xgboost as xgb
from sklearn.model_selection import train_test_split
import spacy
from sklearn.neighbors import KNeighborsClassifier
from spacy import displacy
from gensim.similarities.annoy import AnnoyIndexer
from sklearn.tree import DecisionTreeClassifier
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import nltk
from sklearn.datasets import fetch_20newsgroups
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import matplotlib.gridspec as gridspec
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [3]:
df = pd.read_csv('../input/iba-ml1-final-project/train.csv')
df.head()

,Id,Age,Review_Title,Review,Pos_Feedback_Cnt,Division,Department,Product_Category,Rating,Recommended
0,17274,34,Cute fall/holiday top,Love this top! the quality is magnificent and ...,1,General,Tops,Blouses,5,1
1,5921,35,NaN,NaN,0,General,Tops,Blouses,5,1
2,16479,40,Disappointed,"Sleeves were tight, was difficult to put on ?....",15,General,Tops,Blouses,2,0
3,1925,28,Gorgeous detailing,I never write reviews but this clothe is so fa...,3,General Petite,Clothes,Clothes,5,1
4,5691,39,Cute and comfortable tee!,Love this tshirt! casual but can be clotheed u...,0,General,Tops,Knits,5,1


## Deal with missing value imputations (more detailed version is in EDA notebook)

In [4]:
df['Product_Category'] = df['Product_Category'].fillna(value=df['Product_Category'].mode()[0])
df['Department'] = df['Department'].fillna(value=df['Department'].mode()[0])
df['Division'] = df['Division'].fillna(value=df['Division'].mode()[0])
df['Review'] = df['Review'].fillna(value='nan')
df['Review_Title'] = df['Review_Title'].fillna(value='nan')

In [5]:
df['Rating'] = df['Rating'].apply(lambda s: str(s))
df['Recommended'] = df['Recommended'].apply(lambda s: str(s))

In [20]:
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [36]:
clf = Pipeline(steps=[
    ('vectorization', CountVectorizer()),
#     ('feature_selection', SelectKBest(f_classif, 10000)),
    ('classification', DecisionTreeClassifier(max_depth=10))
])
clf.fit(train_df['Review'], train_df['Recommended'])
print('ROC score',roc_auc_score(test_df['Recommended'], clf.predict(test_df['Review'])))
print("F1 score", f1_score(test_df['Recommended'], clf.predict(test_df['Review']), average="binary", pos_label='1'))

ROC score 0.6074909518937769
F1 score 0.9075630252100839


In [37]:
clf = Pipeline(steps=[
    ('vectorization', CountVectorizer()),
#     ('feature_selection', SelectKBest(f_classif, 10000)),
    ('classification', xgb.XGBClassifier())
])
clf.fit(train_df['Review'], train_df['Recommended'])
print('ROC score',roc_auc_score(test_df['Recommended'], clf.predict(test_df['Review'])))
print("F1 score", f1_score(test_df['Recommended'], clf.predict(test_df['Review']), average="binary", pos_label='1'))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:00:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC score 0.7433899321254525
F1 score 0.9329409291570318


In [45]:
clf = Pipeline(steps=[
    ('vectorization', CountVectorizer()),
    ('feature_selection', SelectKBest(f_classif, 10000)),
    ('classification', xgb.XGBClassifier())
])
clf.fit(train_df[['Review', 'Review_Title']], train_df['Recommended'])
print('ROC score',roc_auc_score(test_df['Recommended'], clf.predict(test_df[['Review', 'Review_Title']])))
print("F1 score", f1_score(test_df['Recommended'], clf.predict(test_df[['Review', 'Review_Title']]), average="binary", pos_label='1'))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass k=10000 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [2, 11304]